In [1]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import seaborn as sns # SEABORN
import scipy
import sklearn
from sklearn.decomposition import PCA
import cv2
import os 


# Ahora ya tenemos las imagenes warpeadas en torno a una forma de referencia (elegida usando Procustres).
## Las cargamos y luego hacemos el metodo basado en  Total Regulation del paper

In [2]:
my_data = pd.read_csv('transformed_landmarks_procustres_total.csv',header = None)
#orden x1xn-y1yn
my_data

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,258.213310,261.555230,267.978110,276.750350,290.519880,310.564040,332.520560,363.126800,397.902110,430.614910,...,399.276060,381.071370,361.327690,343.845490,342.421360,344.785120,356.986140,367.246760,371.105790,367.883230
1,258.260079,261.597406,268.016403,276.783427,290.547142,310.591197,332.539137,363.130286,397.902328,430.608304,...,399.365256,381.114874,361.348807,343.819469,342.387214,344.761274,357.004890,367.318464,371.202325,367.961851
2,258.260079,261.597406,268.016403,276.783427,290.547142,310.591197,332.539137,363.130286,397.902328,430.608304,...,399.365256,381.114874,361.348807,343.819469,342.387214,344.761274,357.004890,367.318464,371.202325,367.961851
3,258.180526,261.525672,267.951280,276.727163,290.500783,310.545016,332.507530,363.124348,397.901963,430.619539,...,399.213509,381.040850,361.312874,343.863722,342.445298,344.801846,356.972986,367.196473,371.038097,367.828110
4,257.731806,261.121102,267.584058,276.409971,290.239485,310.284590,332.329299,363.091027,397.899820,430.682774,...,398.354396,380.621481,361.109119,344.113637,342.773430,345.030917,356.792047,366.505904,370.108661,367.071109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,262.385668,261.062593,266.861863,273.905367,280.123096,297.593762,326.445073,361.495379,398.277240,435.267238,...,382.551530,377.499012,358.141953,345.109024,345.145549,345.665832,354.898678,365.431509,368.794873,369.979998
579,262.459289,261.098531,266.925632,273.945526,280.136290,297.623928,326.464264,361.513025,398.281893,435.260389,...,382.492036,377.440158,358.135715,345.078815,345.125768,345.676078,354.905526,365.450687,368.767366,369.982708
580,262.346649,261.055020,266.834543,273.898203,280.120631,297.541858,326.397965,361.458747,398.250373,435.267297,...,382.482054,377.545025,358.167304,345.151649,345.217583,345.715374,354.863787,365.350292,368.692327,369.956275
581,262.326839,261.042689,266.813314,273.890577,280.132540,297.586126,326.448584,361.489296,398.281757,435.275927,...,382.789229,377.648590,358.193143,345.063979,345.084160,345.576679,354.930971,365.574745,369.051948,370.137988


In [3]:
cant_frames_per_seq = pd.read_csv('cktraining_landmark_FramePerSeq.csv',header = None)
cant_frames_per_seq = np.asarray(cant_frames_per_seq).flatten()


 ### cargamos las imagenes

In [5]:
subject_images = []
warpimgpath = '../ck+warpedtraining'
for i in range(0, len(os.listdir(warpimgpath))):
    subject_images.append('../ck+warpedtraining/' + 'warped_image_procustres_' + str(i)+'.png')
subject_images = np.asarray(subject_images)


In [10]:
### Para empezar, hacemos la descomposición con 1 imagen, luego lo haremos con todas.

La descomposion es f = u + v . Nosotros encontramos u (cartoon) y sabemos v = f- u (texture)

Para hacerla en la primera instancia vamos a hacer el algoritmo del paper

Para eso vamos a hacerlo primero en 1 imagen y luego repilicarlo

Empezamos cargando la imagen.

Luego debemos guardarla por un lado, y por otro aplicarle un low-pass 
filter. Utilizamos el aconsejado  Gaussian filter

In [6]:
img = cv2.imread(subject_images[0])
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow("Texture start", img)

# Pause screen to wait key from user to see result
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite("ejemplotex.png", img)

True

In [ ]:
# the scale parameter sigma es el que utiliza el filtro gaussaino. scale =2 recomendado al inicio.
#La gente percibe con scale 3<s<6
#scale is standart deviation

In [7]:
def iterative_low_pass(imagen, sigma ,times, mod):
    low = scipy.ndimage.gaussian_filter(imagen, sigma, mode = mod)
    high = imagen - low
    for i in range (0, times):
        high = high - scipy.ndimage.gaussian_filter(high, sigma, mode = mod)
        low = imagen - high
    return low, high
    

In [11]:
# lo iteramos 5 veces como dice el paper.
lowpassing,_ = iterative_low_pass(img, 0.5, 100,'reflect')

cv2.imshow("Texture start", lowpassing )

# Pause screen to wait key from user to see result
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("ejemplolowpassim.png", img)
#usamos por ahora mirror mode en todas las convoluvciones como dicho

True

In [12]:
def getgradient(imagen):
    ##sobel_h = scipy.ndimage.sobel(imagen, 0)  # horizontal gradient
    ##sobel_v = scipy.ndimage.sobel(imagen, 1)  # vertical gradient
    sobel_h = cv2.Sobel(img, ddepth=-1, dx=1, dy=0, borderType=cv2.BORDER_REFLECT)
    sobel_v = cv2.Sobel(img, ddepth=-1, dx=0, dy=1, borderType=cv2.BORDER_REFLECT)
    magnitude = np.sqrt(sobel_h**2 + sobel_v**2)
    not_normalized_magnitude = magnitude
    magnitude *= 255.0 / np.max(magnitude)  # normalization
    return not_normalized_magnitude, magnitude

In [14]:
mag_img,_ = getgradient(img)
mag_lowim,_ = getgradient(lowpassing)

mag_img = mag_img.astype('float32')
mag_lowim = mag_lowim.astype('float32')
#usamos la magnitud normalizada- float32 en vez de uint8

In [15]:
local_total_var_img =  cv2.GaussianBlur(mag_img, (5,5), 2., borderType=cv2.BORDER_REFLECT)
local_total_var_lowim = cv2.GaussianBlur(mag_lowim, (5,5), 2., borderType=cv2.BORDER_REFLECT)

In [16]:
diff =  local_total_var_img - local_total_var_lowim

In [17]:
landa = diff / local_total_var_img

/tmp/ipykernel_4554/4277002566.py:1: RuntimeWarning: invalid value encountered in divide
  landa = diff / local_total_var_img


In [19]:

def weigthed_function(data, min_br=0.25, max_br=0.5):
    if data <= min_br:
        return 0.0
    elif data >= max_br:
        return 1.0
    else:
        a = 1 / (max_br - min_br)
        return a * data - a * min_br
        


In [20]:

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))


In [21]:
landa2norm = NormalizeData(np.abs(landa2))


def weigthed_function(data, min_br=0.25, max_br=0.5):
    if data <= min_br:
        return 0.0
    elif data >= max_br:
        return 1.0
    else:
        return data

    
#ahora lo hago asi, luego podemos hacerlo más eficiente con maps o vectorize



NameError: name 'landa2' is not defined

In [22]:
def iterative_low_pass(im, sigma=2. ,times=5, ksize = (5,5)):
    low = cv2.GaussianBlur(im, ksize, sigma, borderType=cv2.BORDER_REFLECT)
    high = im - low
    
    pmin = high.min()
    pmax = high.max()
    
# estamos aca poniendo entre 0 y 255 entre la imagen anterior y la gaussiada
    def calib_minmax(im):
        return (im - im.min()) / (im.max() - im.min()) * (pmax - pmin) + pmin

    
    for i in range (0, times):
        high = high - cv2.GaussianBlur(high, ksize, sigma, borderType=cv2.BORDER_REFLECT)
        high = calib_minmax(high)
        #creo que es lo mismo hacerlo antes o despues del loop lo del low
    low = im - high
    return low, high

def getgradient(img):  
    sobel_h = cv2.Sobel(img, ddepth=-1, dx=1, dy=0, borderType=cv2.BORDER_REFLECT) #hor gradient
    sobel_v = cv2.Sobel(img, ddepth=-1, dx=0, dy=1, borderType=cv2.BORDER_REFLECT) #ver gradient
    magnitude = np.sqrt(sobel_h**2 + sobel_v**2)
    return magnitude

def weigthed_function(data, min_br=0.25, max_br=0.5):
    if data <= min_br:
        return 0.0
    elif data >= max_br:
        return 1.0
    else:
        return data


In [23]:
def cartoon_and_texture_decomposition(img,sig_par, num_pasadas):
    lowpassing,_ = iterative_low_pass(img, sig_par, num_pasadas)
    mag_img = getgradient(img)
    mag_lowim = getgradient(lowpassing)    
    
    mag_img = mag_img.astype('float32')
    mag_lowim = mag_lowim.astype('float32')
    
    local_total_var_img =  cv2.GaussianBlur(mag_img, (5,5), 2., borderType=cv2.BORDER_REFLECT)
    local_total_var_lowim = cv2.GaussianBlur(mag_lowim, (5,5), 2., borderType=cv2.BORDER_REFLECT)
    
    diff =  local_total_var_img - local_total_var_lowim
    landa = diff / local_total_var_img
    #landa = (local_total_var_img - local_total_var_lowim)/local_total_var_img
    #landa2= np.nan_to_num(landa,  nan=0.0, posinf=0.0, neginf=0.0)
    #landa2norm = NormalizeData(np.abs(landa2))
    u = np.zeros((len(landa), len(landa[0])))
    for i in range(0, len(landa)):
        for j in range(0, len(landa[i])):
            u[i][j] = weigthed_function(landa[i][j])*lowpassing[i][j] + (1-weigthed_function(landa[i][j]))*img[i][j]
    v = img - u
    return u , v



In [28]:
def iterative_low_pass(imagen, sigma ,times, mod):
    low = scipy.ndimage.gaussian_filter(imagen, sigma, mode = mod)
    high = imagen - low
    for i in range (0, times):
        high = high - scipy.ndimage.gaussian_filter(high, sigma, mode = mod)
        low = imagen - high
    return low, high

def getgradient(imagen):
    sobel_h = scipy.ndimage.sobel(imagen, 0)  # horizontal gradient
    sobel_v = scipy.ndimage.sobel(imagen, 1)  # vertical gradient
    magnitude = np.sqrt(sobel_h**2 + sobel_v**2)
    not_normalized_magnitude = magnitude
    magnitude *= 255.0 / np.max(magnitude)  # normalization
    return not_normalized_magnitude, magnitude

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def weigthed_function(data, min_br=0.25, max_br=0.5):
    if data <= min_br:
        return 0.0
    elif data >= max_br:
        return 1.0
    else:
        return data


In [29]:
#Vamos a juntarlo todo

def cartoon_and_texture_decomposition(img,sig_par, num_pas):
    lowpassing,_ = iterative_low_pass(img, sig_par, num_pasadas, 'mirror')
    mag_img, normmag_img = getgradient(img)
    mag_lowim, normmag_lowim = getgradient(lowpassing)
    normmag_img = normmag_img.astype('float32')
    normmag_lowim = normmag_lowim.astype('float32')
    local_total_var_img =  scipy.ndimage.gaussian_filter(normmag_img, 2, mode = 'mirror')
    local_total_var_lowim = scipy.ndimage.gaussian_filter(normmag_lowim, 2, mode = 'mirror')
    landa = (local_total_var_img - local_total_var_lowim)/local_total_var_img
    landa2= np.nan_to_num(landa,  nan=0.0, posinf=0.0, neginf=0.0)
    landa2norm = NormalizeData(np.abs(landa2))
    u = np.zeros((len(landa2norm), len(landa2norm[0])))
    for i in range(0, len(landa2norm)):
        for j in range(0, len(landa2norm[i])):
            u[i][j] = weigthed_function(landa2norm[i][j])*lowpassing[i][j] + (1-weigthed_function(landa2norm[i][j]))*img[i][j]
    v = img - u
    return u , v


In [30]:
im = cv2.imread(subject_images[0],0)

u, v = cartoon_and_texture_decomposition(im, 2,5, 50)

NameError: name 'num_pasadas' is not defined

In [19]:
cv2.imshow("Texture cartoon", u)

# Pause screen to wait key from user to see result
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("ejemplou.png", u)
cv2.imwrite("ejemplov.png", v)

True

In [37]:
umean = u /np.mean(u)

In [38]:
subjects_images = '../ck+warped'

first_images = []
first_frames = []
total = 0
first_frames.append(my_data.iloc[total])#first frame of first secuence
first_images.append(subjects_images + '/' + 'warped_image_procustres_' + str(total)+ '.png')

for i in range(0, len(cant_frames_per_seq)-1):
    total = total+cant_frames_per_seq[i]
    first_frames.append(my_data.iloc[total])
    first_images.append(subjects_images + '/' + 'warped_image_procustres_' + str(total)+ '.png')


first_frames = np.asarray( first_frames, dtype=object)   
first_images = np.asarray( first_images, dtype=object)   

intercalated_first_frames = []
for fframe in first_frames:
    xaxis = fframe[0: int(len(first_frames[0])/2)]
    yaxis = fframe[int(len(first_frames[0])/2):len(first_frames[0])]
    lan =[]
    for (a, b) in zip(xaxis, yaxis):
        lan.append(np.asarray([a,b]))        
    intercalated_first_frames.append( np.asarray(lan))
intercalated_first_frames = np.asarray(intercalated_first_frames)


first_frame = intercalated_first_frames

In [45]:
#falta lo del hull

In [ ]:
cartoon = []
texture = []
for ima in subject_images:    
    imaactual =cv2.imread(ima)
    imaactual = cv2.cvtColor(imaactual, cv2.COLOR_BGR2GRAY)
    u, v = cartoon_and_texture_decomposition(imaactual, 2,5)
    cartoon.append(u)
    texture.append(v)
    
cartoon = np.asarray(cartoon)
texture = np.asarray(texture)

/tmp/ipykernel_10998/2381606023.py:11: RuntimeWarning: divide by zero encountered in divide
  landa = (local_total_var_img - local_total_var_lowim)/local_total_var_img
/tmp/ipykernel_10998/2381606023.py:11: RuntimeWarning: invalid value encountered in divide
  landa = (local_total_var_img - local_total_var_lowim)/local_total_var_img


In [44]:
for uva in cartoon:
    uva = uva - np.mean(uva)
    

ind =0
my_cartoon_diff =[]
for i in range(0, len(cant_frames_per_seq)):#51
    cant_frames = cant_frames_per_seq[i]
    frame_zero = cartoon.iloc[ind]    
    for j in range(ind, ind + cant_frames):
        my_cartoon_diff.append(cartoon.iloc[j]- cartoon.iloc[ind]) #no dice nada de abs
    ind = ind + cant_frames
    
my_cartoon_diff = np.asarray(my_cartoon_diff)

(490, 640)

In [ ]:
my_df = pd.DataFrame(my_cartoon_diff.flatten())
my_df.to_csv('my_cartoon_diff.csv',header = False, index= False)
my_df = pd.DataFrame(texture.flatten())
my_df.to_csv('my_texture.csv',header = False, index= False)

In [ ]:
#Aca hacemos PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
my_data_forPCA = pd.read_csv('my_cartoon_diff.csv',header = None)
my_data_forPCA
std_scaler = StandardScaler()
scaled_landDiff = std_scaler.fit_transform(my_data_forPCA)
scaled_landDiff.shape
pca = PCA(n_components=29)
pca.fit_transform(scaled_landDiff)
b_shape = pca.fit_transform(scaled_landDiff)
b_shape.shape

In [ ]:
print(sum(pca.explained_variance_ratio_))

In [ ]:
b_shape
pca_var=np.round(pca.explained_variance_ratio_* 100, decimals=1 )
labels = ['PC'+ str(s) for s in range(1, len(pca_var)+1)]

plt.bar(x= range(1, len(pca_var)+1), height = pca_var, tick_label= labels)
plt.ylabel("Percentage of Variance")
plt.xlabel("Principal Components")
plt.title("Scree Plot")
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
my_data_forPCA = pd.read_csv('my_texture.csv',header = None)
my_data_forPCA
std_scaler = StandardScaler()
scaled_landDiff = std_scaler.fit_transform(my_data_forPCA)
scaled_landDiff.shape
pca = PCA(n_components=29)
pca.fit_transform(scaled_landDiff)
b_shape = pca.fit_transform(scaled_landDiff)
b_shape.shape

In [ ]:
print(sum(pca.explained_variance_ratio_))

In [ ]:
b_shape
pca_var=np.round(pca.explained_variance_ratio_* 100, decimals=1 )
labels = ['PC'+ str(s) for s in range(1, len(pca_var)+1)]

plt.bar(x= range(1, len(pca_var)+1), height = pca_var, tick_label= labels)
plt.ylabel("Percentage of Variance")
plt.xlabel("Principal Components")
plt.title("Scree Plot")
plt.show()

In [40]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d
#Aca es para el hull, ahora lo hacemos con toda la imagen, sino hay que ver primero
points = first_frame[0][0:26]
hull = ConvexHull(points)
hull

#for i in range(0, len(first_frame)):   
 #   points = first_frame[i][0:26]
 #   hull = ConvexHull(points)

In [31]:
#cv2.imshow("cartoon", u)
cv2.imshow("texture", v)
#cv2.imahow('original', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import tensorflow as tf


img = cv2.imread(subject_images[0])
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_tensor = tf.convert_to_tensor(img_rgb, dtype=tf.float32)
img_gray = tf.image.rgb_to_grayscale(img_tensor)

tot_var = tf.image.total_variation(img_gray) 
tot_var.numpy()

In [ ]:
tf.print(tot_var, [tot_var])